In [1]:
import pandas as pd
import os
import datetime as dt
import pickle
import numpy as np
import nbimporter
from exploratory_analysis import load_obj, returnDataOnDate, returnVolumeAndPriceChange
import warnings
warnings.filterwarnings('ignore')

In [2]:
DAYS_TO_LOOK_FORWARD = 7  # number of days after trade to look for max/min price (see exploratory_analysis)
DAYS_TO_LOOK_BACK = 1  # number of days before trade to look for price trends / related insider trades

In [3]:
train_data = pd.read_csv('data/insiderDat_June_clean.csv')
train_data.head()

,FilingDate,TradeDate,Ticker,CompanyName,InsiderName,Title,TradeType,Price,Qty,Owned,DeltaOwn,Value
0,2021-06-01,2021-06-01,ALK,"Alaska Air Group, Inc.",Tilden Bradley D,COB,S - Sale,69.85,-6500.0,133500.0,-5.0,-454017.0
1,2021-06-01,2021-06-01,CVCY,Central Valley Community Bancorp,Kim James J,EVP,P - Purchase,16.43,619.0,14396.0,4.0,10173.0
2,2021-06-01,2021-06-01,OPOF,Old Point Financial Corp,Keefe William F,Dir,P - Purchase,23.00,200.0,7900.0,3.0,4600.0
3,2021-06-01,2021-06-01,CARE,"Carter Bankshares, Inc.",Adams Arthur Loran,"EVP, Dir OF REG RISK MNGT",P - Purchase,15.41,19.0,3041.0,1.0,293.0
4,2021-06-01,2021-06-01,CARE,"Carter Bankshares, Inc.",Bird Michael R,Dir,P - Purchase,15.41,32.0,6514.0,0.0,493.0


In [4]:
train_data['FilingDate'] = pd.to_datetime(train_data['FilingDate']).dt.date
train_data['TradeDate'] = pd.to_datetime(train_data['TradeDate']).dt.date

In [5]:
historicDat = load_obj('historicDat')
historicDat

{'IBP':                   Open        High         Low       Close   Adj Close  Volume
 Date                                                                          
 2021-06-01  119.379997  120.970001  118.169998  120.449997  119.582275  303100
 2021-06-02  119.849998  119.849998  117.129997  117.639999  116.792519  300600
 2021-06-03  116.800003  117.800003  114.470001  117.220001  116.375542  280000
 2021-06-04  118.669998  119.900002  115.779999  119.709999  118.847603  137400
 2021-06-07  120.059998  120.489998  118.620003  119.300003  118.440567  125400
 ...                ...         ...         ...         ...         ...     ...
 2021-11-22  138.610001  141.100006  136.270004  138.889999  136.382538   98700
 2021-11-23  138.679993  140.070007  136.690002  139.410004  136.893143  120700
 2021-11-24  137.919998  138.929993  136.820007  137.779999  135.292572  127400
 2021-11-26  133.399994  135.429993  130.509995  133.169998  130.765793  141900
 2021-11-29  135.330002  135.3300

In [6]:
allTickers = set()

# updates the set with all of the dictonary's keys
allTickers.update(historicDat)

In [24]:
train_data[['NumTrades','NumTradesCAT','TradeToFileTime','TradeToFileTimeCAT', \
            '%VolumeChange','%VolumeChangeCAT','%FuturePriceChange']] = None

startDate = dt.datetime.strptime('2021-06-01', '%Y-%m-%d').date()
endDate = dt.datetime.strptime('2021-06-30', '%Y-%m-%d').date()
delta = endDate - startDate

for trade in train_data.iterrows():
    tradeNum = trade[0]
    tradeTick = trade[1]['Ticker']
    tradeDate = trade[1]['TradeDate']
    tradeFileDate = trade[1]['FilingDate']
    
    # skip the first DAYS_TO_LOOK_BACK days so we have data to look back at
    if (tradeFileDate - dt.timedelta(days=DAYS_TO_LOOK_BACK)) < startDate:
        continue
    
    
    # compute percentage change in shares owned by insider
    owned = train_data.at[tradeNum, 'Owned']
    shareChange = train_data.at[tradeNum, 'Qty']
    train_data.at[tradeNum, 'DeltaOwn'] = 100*shareChange / (owned-shareChange)
    
    
    # compute total value of insider's trade
    train_data.at[tradeNum, 'Value'] = shareChange*train_data.at[tradeNum, 'Price']
    
    
    # compute total value of insider's shares
    # TODO
    
    
    # compute and categorize time gaps between trades and filings
    tradeToFileTime = (tradeFileDate - tradeDate).days
    train_data.at[tradeNum, 'TradeToFileTime'] = tradeToFileTime
    
    if tradeToFileTime == 0:
        train_data.at[tradeNum, 'TradeToFileTimeCAT'] = '0'
    elif tradeToFileTime == 1:
        train_data.at[tradeNum, 'TradeToFileTimeCAT'] = '1'
    else:
        train_data.at[tradeNum, 'TradeToFileTimeCAT'] = '>1'


    # compute and categorize the number of same-ticker trades in the last DAYS_TO_LOOK_BACK days
    numTickerTrades = train_data.loc[(train_data['Ticker'] == tradeTick)
                                    & (train_data['FilingDate'] <= tradeFileDate)
                                    & (train_data['FilingDate'] 
                                       >= tradeFileDate-dt.timedelta(days=DAYS_TO_LOOK_BACK))].shape[0]

    train_data.at[tradeNum, 'NumTrades'] = numTickerTrades
    if numTickerTrades == 1:
        train_data.at[tradeNum, 'NumTradesCAT'] = '1'
    elif numTickerTrades == 2:
        train_data.at[tradeNum, 'NumTradesCAT'] = '2'
    else:
        train_data.at[tradeNum, 'NumTradesCAT'] = '>2'


    # compute and categorize the percentage volume change in the last DAYS_TO_LOOK_BACK days
    # compute the most significant closing price percentage change in the next DAYS_TO_LOOK_FORWARD days
    percentChangeVol, percentChangePrice = returnVolumeAndPriceChange(historicDat, tradeTick, tradeFileDate, 
                                                                     DAYS_TO_LOOK_FORWARD, DAYS_TO_LOOK_BACK)
    train_data.at[tradeNum, '%FuturePriceChange'] = percentChangePrice
    
    train_data.at[tradeNum, '%VolumeChange'] = percentChangeVol
    if percentChangeVol <= -10:
        train_data.at[tradeNum, '%VolumeChangeCAT'] = '<-10%'
    elif percentChangeVol > -10 and percentChangeVol <= 0:
        train_data.at[tradeNum, '%VolumeChangeCAT'] = '-10 to 0%'
    elif percentChangeVol > 0 and percentChangeVol <= 10:
        train_data.at[tradeNum, '%VolumeChangeCAT'] = '0 to 10%'
    else:
        train_data.at[tradeNum, '%VolumeChangeCAT'] = '>10%'
    
    
    

    '''
    refval = historicDat[tradeTick]['Close']  #tickerDat[t][tickerDat[t].index == '2021-06-01']['Open'][0]
    refDat['ref'][refDat.index == t] = refval
    maxJuly = max(tickerDat[t][tickerDat[t].index.isin(June_July_days)]['Open'])
    perChange = (maxJuly-refval)/refval

    refDat['perChange'][refDat.index == t] = perChange
    '''
    #except:
    #    print(f'This trade was not processed: {tradeNum}')

In [25]:
train_data.loc[train_data['FilingDate'] == dt.datetime.fromisoformat('2021-06-02').date()]

,FilingDate,TradeDate,Ticker,CompanyName,InsiderName,Title,TradeType,Price,Qty,Owned,DeltaOwn,Value,NumTrades,NumTradesCAT,TradeToFileTime,TradeToFileTimeCAT,%VolumeChange,%VolumeChangeCAT,%FuturePriceChange
31,2021-06-02,2021-06-02,MMS,"Maximus, Inc.",Ruddy Raymond B,Dir,S - Sale,90.87,-24820.0,103648.0,-19.0,-2255339.0,1,1,0,0,20.0,>10%,-0.978882
32,2021-06-02,2021-06-02,HDSN,Hudson Technologies Inc /ny,Houghton Kathleen,"VP - Sales, Marketing",S - Sale+OE,3.00,-3582.0,0.0,-100.0,-10740.0,2,2,0,0,-62.642529,<-10%,5.666669
33,2021-06-02,2021-06-01,WRLD,World Acceptance Corp,Way Charles D,Dir,S - Sale+OE,163.75,-5000.0,14989.0,-25.0,-818735.0,1,1,1,1,-19.626168,<-10%,-6.34794
34,2021-06-02,2021-06-01,HDSN,Hudson Technologies Inc /ny,Abbatecola Vincent P,Dir,S - Sale,2.95,-16298.0,116928.0,-12.0,-48079.0,2,2,1,1,-62.642529,<-10%,5.666669
35,2021-06-02,2021-06-01,TRST,Trustco Bank Corp N Y,Leonard Robert M,"EVP, CHIEF RISK OFFICER",P - Purchase,38.00,604.0,23925.0,3.0,22952.0,1,1,1,1,-39.915966,<-10%,-3.844097
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223,2021-06-02,2021-06-01,INSW,"International Seaways, Inc.",Pribor Jeffrey,"SVP,CFO, Treasurer",S - Sale,20.25,-1000.0,47164.0,-2.0,-20249.0,1,1,1,1,19.210841,>10%,3.726081
224,2021-06-02,2021-06-01,SI,Silvergate Capital Corp,Brassfield Karen F.,Dir,S - Sale+OE,117.30,-4000.0,16298.0,-20.0,-469200.0,1,1,1,1,-39.324668,<-10%,-9.349812
225,2021-06-02,2021-06-01,PANW,Palo Alto Networks Inc,Compeau Jean,"Deputy CFO, CAO",S - Sale,360.07,-2498.0,22112.0,-10.0,-899442.0,1,1,1,1,6.499322,0 to 10%,-1.828503
226,2021-06-02,2021-06-01,VIR,"Vir Biotechnology, Inc.",Virgin Herbert,"EVP, Research, CSO",S - Sale+OE,42.51,-1375.0,91700.0,-1.0,-58451.0,1,1,1,1,113.686227,>10%,6.609709


In [27]:
train_data.to_csv('data/training_data.csv')